In [47]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [48]:
def developer(desarrollador):

    df = pd.read_parquet('./Datasets/developer_endpoint.parquet')
    df = df[df['developer'] == desarrollador]
    df['Año'] = df['release_date'].dt.year
    df_year = df.groupby('Año').size().reset_index(name='Cantidad de Items')
    df_free = df.groupby('Año')['free'].sum().reset_index(name='free')
    df = pd.merge(df_year, df_free, on='Año')
    df['Contenido Free'] = df['free'] / df['Cantidad de Items'] * 100
    df.drop(columns=['free'], inplace=True)
    df = df.to_dict('records')
    return df

In [49]:
developer('Valve')

[{'Año': 1998, 'Cantidad de Items': 1, 'Contenido Free': 0.0},
 {'Año': 1999, 'Cantidad de Items': 1, 'Contenido Free': 0.0},
 {'Año': 2000, 'Cantidad de Items': 2, 'Contenido Free': 0.0},
 {'Año': 2001, 'Cantidad de Items': 1, 'Contenido Free': 0.0},
 {'Año': 2003, 'Cantidad de Items': 1, 'Contenido Free': 0.0},
 {'Año': 2004, 'Cantidad de Items': 5, 'Contenido Free': 0.0},
 {'Año': 2006, 'Cantidad de Items': 2, 'Contenido Free': 0.0},
 {'Año': 2007, 'Cantidad de Items': 3, 'Contenido Free': 33.33333333333333},
 {'Año': 2008, 'Cantidad de Items': 1, 'Contenido Free': 0.0},
 {'Año': 2009, 'Cantidad de Items': 1, 'Contenido Free': 0.0},
 {'Año': 2010, 'Cantidad de Items': 2, 'Contenido Free': 50.0},
 {'Año': 2011, 'Cantidad de Items': 1, 'Contenido Free': 0.0},
 {'Año': 2012, 'Cantidad de Items': 1, 'Contenido Free': 0.0},
 {'Año': 2016, 'Cantidad de Items': 1, 'Contenido Free': 100.0},
 {'Año': 2017, 'Cantidad de Items': 2, 'Contenido Free': 0.0}]

In [64]:
def game_recommendation(item_id):
    item_id = str(item_id)
    df = pd.read_parquet('./Datasets/game_recommendation.parquet')
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(df['features'])
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    if item_id not in df['item_id'].values:
        return 'ID no encontrado'
    idx = df[df['item_id'] == item_id].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    recommended_games = df['title'].iloc[[i[0] for i in sim_scores]].tolist()
    return {"recommended_games": recommended_games}

In [65]:
game_recommendation(730)

{'recommended_games': ['Insurgency',
  'Counter-Strike: Source',
  'Arma 3',
  'TankZone Battle',
  'Arma 2: Operation Arrowhead']}